## Imports

In [1]:
%reset -f
import pandas
import configparser
import os
import numpy as np
# %load_ext autoreload
# %autoreload 2
import sys
# sys.path.append('/home')

import logging
# 设置日志记录器的级别为 WARNING（大于或等于WARNING的日志会显示）
logging.basicConfig(level=logging.WARNING)

from baseline_int.graph_plot.barchart import BarChart

%matplotlib inline
import matplotlib
import re
from itertools import product

import warnings
warnings.filterwarnings('ignore')

import baseline_int.src.benchmarks.benchmarks as benchmarks
from baseline_int.src.simulator.stats import Stats
from baseline_int.src.simulator.simulator import Simulator
from baseline_int.src.sweep.sweep import SimulatorSweep, check_pandas_or_run
from baseline_int.src.utils.utils import *
from baseline_int.src.optimizer.optimizer import optimize_for_order, get_stats_fast

import dnnweaver2

from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import math
import tqdm
plt.rcParams['font.family'] = ['serif']
plt.rcParams['font.serif'] = ['Times New Roman']
font = { 'size'   : 18}
plt.rc('font', **font)
color_bar1 = ["#7CBD3D", "#33B7FF", "#FF6F66", "#E3A735", "#E96A66", "#457B9D", "#66AB9E"]
color_bar2 = ["#2a9d8f", "#F6BD60", "#457b9d", "#F07167", "#BFA6A2", "#A1CD5A", "#ED7D31"]
color_bar4 = ["#d6d6d7", "#b9b9b9", "#848484", "#50595a", "#383838", "#a31f33"]
# hatches = ['//', 'xx', '..', '\\', '|', '-', '+', 'x', 'o', 'O', '.', '*']
hatches = ['//', 'xx', '\\', '-', '+', 'x', 'o', 'O', '.', '*']

In [2]:
# dataframe转换到仿真器状态stats
def df_to_stats(df):
    stats = Stats()
    stats.total_cycles = float(df['Cycles'])
    stats.ptb_cycles = float(df['PTB-Cycles'])
    stats.ps5_cycles = float(df['PS5-Cycles'])
    stats.mem_stall_cycles = float(df['Memory wait cycles'])
    stats.reads['act'] = float(df['IBUF Read'])
    stats.reads['out'] = float(df['OBUF Read'])
    stats.reads['wgt'] = float(df['WBUF Read'])
    stats.reads['dram'] = float(df['DRAM Read'])
    stats.writes['act'] = float(df['IBUF Write'])
    stats.writes['out'] = float(df['OBUF Write'])
    stats.writes['wgt'] = float(df['WBUF Write'])
    stats.writes['dram'] = float(df['DRAM Write'])
    return stats

In [3]:
results_dir = './results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

fig_dir = './fig'
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)

### Create a simulator object using the configuration from PS4-Eyeriss (ps4_e_conf.ini)

In [4]:
# 定义模拟器中需要扫描的列
sim_sweep_columns = ['N', 'M',
        'Max Precision (bits)', 'Min Precision (bits)',
        'Network', 'Layer',
        'Cycles', 'Memory wait cycles',
        'WBUF Read', 'WBUF Write',
        'OBUF Read', 'OBUF Write',
        'IBUF Read', 'IBUF Write',
        'DRAM Read', 'DRAM Write',
        'Bandwidth (bits/cycle)',
        'WBUF Size (bits)', 'OBUF Size (bits)', 'IBUF Size (bits)',
        'Timesteps', 'in_sparsity_util', 'out_sparsity_util', 'TW', 'Speed']

# 参数替换
def replace_in_config_file(config_file, key, val):
    with open(config_file, 'r') as f:
        content = f.read()
    content = re.sub(r'^{} = (\d+)'.format(key), '{} = {}'.format(key, val), content, flags=re.MULTILINE)
    with open(config_file, 'w') as f:
        f.write(content)

# 仿真过程。
def sim(TW, N, M, Speed=True):
    # Speed 变量控制是否对时间窗做加速
    # BitFusion configuration file
    config_file = 'ps5_e_conf_test.ini'
    # replace "a = (\d+)"" in config_file to "a = N"
    replace_in_config_file(config_file=config_file, key='a', val=N)  # 以a输出N，以c输出M
    replace_in_config_file(config_file=config_file, key='c', val=M)

    timesteps = 100

    syn_fn = 'systolic_array_synth_snn.csv'  # 脉动阵列仿真文件

    # Create simulator object
    verbose = False  # 设置输出信息类型
    ps5_e_sim = Simulator(config_file, verbose)  # 建立仿真器
    # ps5_e_energy_costs = ps5_e_sim.get_energy_cost(syn_fn)  # 计算能耗
    # print(ps5_e_sim)

    ps5_e_sim_sweep_csv = os.path.join(results_dir, 'ps5-sim-sweep-test.csv')  # 扫描结果文件
    # if os.path.exists(ps4_e_sim_sweep_csv):
    #     os.remove(ps4_e_sim_sweep_csv)
    if os.path.exists(ps5_e_sim_sweep_csv):  # 读取文件 或 新建
        ps5_e_sim_sweep_df = pandas.read_csv(ps5_e_sim_sweep_csv)
    else:
        ps5_e_sim_sweep_df = pandas.DataFrame(columns=sim_sweep_columns)
    print('Got BitFusion Eyeriss, Numbers')

    ps5_e_results = check_pandas_or_run(ps5_e_sim, ps5_e_sim_sweep_df,
            ps5_e_sim_sweep_csv, batch_size=timesteps, config_file=config_file, TW=TW, list_bench=benchmarks.snn_benchlist, Speed=Speed)  # 运行仿真
    return ps5_e_results


def generate_col_row_pairs(PE_number):
    result = []
    max_pow = 12  # 因为 2^12 = 4096 是最大符合要求的 PE_number

    for col_exp in range(max_pow + 1):
        for row_exp in range(max_pow + 1):
            col_number = 2 ** col_exp
            row_number = 2 ** row_exp
            if col_number * row_number <= PE_number:
                result.append([col_number, row_number])
    
    # sim_stop_list = [[1, 1], [1, 2], [2, 1], [2, 2], [1, 4], [1, 8], [1, 16], [1, 32], [2, 4], [2, 8], [2, 16], [4, 1], [4, 2], [4, 4], 
    #                 [4, 8], [8, 1], [8, 2], [8, 4], [16, 1], [16, 2], [32, 1], [1, 1024], [1, 2048], [1, 4096], [2, 512], [2, 1024], 
    #                 [2, 2048], [4, 256], [4, 512], [4, 1024], [8, 128], [8, 256], [8, 512], [16, 64], [16, 128], [16, 256], [32, 64], 
    #                 [32, 128], [64, 16], [64, 32], [128, 8], [128, 16], [128, 32], [256, 4], [256, 8], [256, 16], [512, 1], [512, 2], 
    #                 [512, 4], [512, 8], [1024, 1], [1024, 2], [1024, 4], [2048, 1], [2048, 2], [4096, 1]]
    sim_stop_list = []
    filtered_result = [elem for elem in result if elem not in sim_stop_list]

    sim_append_list = [[64, 64], [128, 128], [256, 256]]

    filtered_result.extend(sim_append_list)

    return filtered_result

In [5]:
def generate_col_row_pairs(PE_number):
    result = []
    for M in [1, 2, 4, 8, 16, 32, 64, 128, 256]:
        for N in [1, 2, 4, 8, 16, 32, 64, 128, 256]:
            result.append([M, N])
    return result

In [6]:
total_PE_number = 2048
TW = 8
col_row_combine = generate_col_row_pairs(total_PE_number)

In [7]:
sim_result = sim(TW, col_row_combine[0][1], col_row_combine[0][0])
for combine in tqdm.tqdm(col_row_combine[1:]):
    PE_column_number = combine[0]
    PE_row_number = combine[1]
    per_combine_sim_result = sim(TW, PE_row_number, PE_column_number)
    sim_result = pd.concat([sim_result, per_combine_sim_result], axis=0)
sim_result.to_csv('baseline_test.csv', index=False)

Got BitFusion Eyeriss, Numbers


INFO:baseline_int.src.sweep.sweep.Simulator:Simulating Benchmark: CIFAR10SNN
INFO:baseline_int.src.sweep.sweep.Simulator:N x M = 1 x 1
INFO:baseline_int.src.sweep.sweep.Simulator:Max Precision (bits): 8
INFO:baseline_int.src.sweep.sweep.Simulator:Min Precision (bits): 8
INFO:baseline_int.src.sweep.sweep.Simulator:Batch size: 100
INFO:baseline_int.src.sweep.sweep.Simulator:Bandwidth (bits/cycle): 1024
INFO:baseline_int.src.sweep.sweep.Simulator:Simulating Benchmark: DVS128Gesture
INFO:baseline_int.src.sweep.sweep.Simulator:N x M = 1 x 1
INFO:baseline_int.src.sweep.sweep.Simulator:Max Precision (bits): 8
INFO:baseline_int.src.sweep.sweep.Simulator:Min Precision (bits): 8
INFO:baseline_int.src.sweep.sweep.Simulator:Batch size: 100
INFO:baseline_int.src.sweep.sweep.Simulator:Bandwidth (bits/cycle): 1024
INFO:baseline_int.src.sweep.sweep.Simulator:Simulating Benchmark: ImageNetSNN
INFO:baseline_int.src.sweep.sweep.Simulator:N x M = 1 x 1
INFO:baseline_int.src.sweep.sweep.Simulator:Max Preci

Got BitFusion Eyeriss, Numbers


INFO:baseline_int.src.sweep.sweep.Simulator:Simulating Benchmark: CIFAR10SNN
INFO:baseline_int.src.sweep.sweep.Simulator:N x M = 2 x 1
INFO:baseline_int.src.sweep.sweep.Simulator:Max Precision (bits): 8
INFO:baseline_int.src.sweep.sweep.Simulator:Min Precision (bits): 8
INFO:baseline_int.src.sweep.sweep.Simulator:Batch size: 100
INFO:baseline_int.src.sweep.sweep.Simulator:Bandwidth (bits/cycle): 1024
INFO:baseline_int.src.sweep.sweep.Simulator:Simulating Benchmark: DVS128Gesture
INFO:baseline_int.src.sweep.sweep.Simulator:N x M = 2 x 1
INFO:baseline_int.src.sweep.sweep.Simulator:Max Precision (bits): 8
INFO:baseline_int.src.sweep.sweep.Simulator:Min Precision (bits): 8
INFO:baseline_int.src.sweep.sweep.Simulator:Batch size: 100
INFO:baseline_int.src.sweep.sweep.Simulator:Bandwidth (bits/cycle): 1024
INFO:baseline_int.src.sweep.sweep.Simulator:Simulating Benchmark: ImageNetSNN
INFO:baseline_int.src.sweep.sweep.Simulator:N x M = 2 x 1
INFO:baseline_int.src.sweep.sweep.Simulator:Max Preci

Got BitFusion Eyeriss, Numbers


...

error
error
error
error


INFO:baseline_int.src.sweep.sweep.Simulator:Simulating Benchmark: ImageNetSNN
INFO:baseline_int.src.sweep.sweep.Simulator:N x M = 256 x 256
INFO:baseline_int.src.sweep.sweep.Simulator:Max Precision (bits): 8
INFO:baseline_int.src.sweep.sweep.Simulator:Min Precision (bits): 8
INFO:baseline_int.src.sweep.sweep.Simulator:Batch size: 100
INFO:baseline_int.src.sweep.sweep.Simulator:Bandwidth (bits/cycle): 1024


error
error
error
error
error
error
error
error
error
error
error
error
error
error


100%|██████████| 80/80 [1:21:59<00:00, 61.50s/it]

error


In [8]:
sim_result = sim(TW, col_row_combine[0][1], col_row_combine[0][0], False)
for combine in tqdm.tqdm(col_row_combine[1:]):
    PE_column_number = combine[0]
    PE_row_number = combine[1]
    per_combine_sim_result = sim(TW, PE_row_number, PE_column_number, False)
    sim_result = pd.concat([sim_result, per_combine_sim_result], axis=0)
sim_result.to_csv('baseline.csv', index=False)

Got BitFusion Eyeriss, Numbers


INFO:baseline_int.src.sweep.sweep.Simulator:Simulating Benchmark: CIFAR10SNN
INFO:baseline_int.src.sweep.sweep.Simulator:N x M = 1 x 1
INFO:baseline_int.src.sweep.sweep.Simulator:Max Precision (bits): 8
INFO:baseline_int.src.sweep.sweep.Simulator:Min Precision (bits): 8
INFO:baseline_int.src.sweep.sweep.Simulator:Batch size: 100
INFO:baseline_int.src.sweep.sweep.Simulator:Bandwidth (bits/cycle): 1024
INFO:baseline_int.src.sweep.sweep.Simulator:Simulating Benchmark: DVS128Gesture
INFO:baseline_int.src.sweep.sweep.Simulator:N x M = 1 x 1
INFO:baseline_int.src.sweep.sweep.Simulator:Max Precision (bits): 8
INFO:baseline_int.src.sweep.sweep.Simulator:Min Precision (bits): 8
INFO:baseline_int.src.sweep.sweep.Simulator:Batch size: 100
INFO:baseline_int.src.sweep.sweep.Simulator:Bandwidth (bits/cycle): 1024
INFO:baseline_int.src.sweep.sweep.Simulator:Simulating Benchmark: ImageNetSNN
INFO:baseline_int.src.sweep.sweep.Simulator:N x M = 1 x 1
INFO:baseline_int.src.sweep.sweep.Simulator:Max Preci

Got BitFusion Eyeriss, Numbers


INFO:baseline_int.src.sweep.sweep.Simulator:Simulating Benchmark: CIFAR10SNN
INFO:baseline_int.src.sweep.sweep.Simulator:N x M = 2 x 1
INFO:baseline_int.src.sweep.sweep.Simulator:Max Precision (bits): 8
INFO:baseline_int.src.sweep.sweep.Simulator:Min Precision (bits): 8
INFO:baseline_int.src.sweep.sweep.Simulator:Batch size: 100
INFO:baseline_int.src.sweep.sweep.Simulator:Bandwidth (bits/cycle): 1024
INFO:baseline_int.src.sweep.sweep.Simulator:Simulating Benchmark: DVS128Gesture
INFO:baseline_int.src.sweep.sweep.Simulator:N x M = 2 x 1
INFO:baseline_int.src.sweep.sweep.Simulator:Max Precision (bits): 8
INFO:baseline_int.src.sweep.sweep.Simulator:Min Precision (bits): 8
INFO:baseline_int.src.sweep.sweep.Simulator:Batch size: 100
INFO:baseline_int.src.sweep.sweep.Simulator:Bandwidth (bits/cycle): 1024
INFO:baseline_int.src.sweep.sweep.Simulator:Simulating Benchmark: ImageNetSNN
INFO:baseline_int.src.sweep.sweep.Simulator:N x M = 2 x 1
INFO:baseline_int.src.sweep.sweep.Simulator:Max Preci

Got BitFusion Eyeriss, Numbers


...

error
error
error
error


INFO:baseline_int.src.sweep.sweep.Simulator:Simulating Benchmark: ImageNetSNN
INFO:baseline_int.src.sweep.sweep.Simulator:N x M = 256 x 256
INFO:baseline_int.src.sweep.sweep.Simulator:Max Precision (bits): 8
INFO:baseline_int.src.sweep.sweep.Simulator:Min Precision (bits): 8
INFO:baseline_int.src.sweep.sweep.Simulator:Batch size: 100
INFO:baseline_int.src.sweep.sweep.Simulator:Bandwidth (bits/cycle): 1024


error
error
error
error
error
error
error
error
error
error
error
error
error
error


100%|██████████| 80/80 [1:19:23<00:00, 59.54s/it]

error


In [ ]:
sim_result = sim(TW, 64, 1)
sim_result.to_csv('baseline_test.csv', index=False)

In [ ]:
sim_result = sim(TW, 64, 1, False)
sim_result.to_csv('baseline.csv', index=False)

In [ ]:
sim_result = sim(TW, 2, 256, False)
sim_result.to_csv('baseline.csv', index=False)

In [ ]:
TW = 1
sim_result = sim(TW, col_row_combine[0][1], col_row_combine[0][0])
for combine in tqdm.tqdm(col_row_combine[1:]):
    PE_column_number = combine[0]
    PE_row_number = combine[1]
    per_combine_sim_result = sim(TW, PE_row_number, PE_column_number)
    sim_result = pd.concat([sim_result, per_combine_sim_result], axis=0)
sim_result.to_csv('baseline_sim_batch_tiling_allmodel_TW1.csv', index=False)

## Performance Scaling

以上实验结果获得了baseline的性能，下面粗略的通过稀疏率估计最终性能。

通过读取trace文件获取每一层的input/output稀疏利用率，从而计算speedup和energy efficiency。

TODO: 存在问题在于不同TW需要更多的on-chip buffer，这个暂时无法评估。

### Input sparsity for PTB

In [ ]:
# 定义ps4配置文件、batchsize、TW、脉动阵列配置文件
config_file = 'ps5_e_conf_test.ini'
batch_size = 100
TW = 8
syn_fn = 'systolic_array_synth_snn.csv'
verbose = False

# 根据ps4配置文件建立仿真器
ps4_e_sim = Simulator(config_file, verbose)

# 结合脉动阵列配置文件计算能耗
ps4_e_energy_costs = ps4_e_sim.get_energy_cost(syn_fn)
print(ps4_e_sim)

# 输出能耗
energy_tuple = ps4_e_energy_costs
print('')
print('*'*50)
print(energy_tuple)

# 创建结果保存文件，存放在不同PE配置下神经网络每一层的扫描结果
ps4_e_sim_sweep_csv = os.path.join(results_dir, 'ps4-sim-sweep.csv')
# if os.path.exists(ps4_e_sim_sweep_csv):
#     os.remove(ps4_e_sim_sweep_csv)
if os.path.exists(ps4_e_sim_sweep_csv):
    ps4_e_sim_sweep_df = pandas.read_csv(ps4_e_sim_sweep_csv)
else:
    ps4_e_sim_sweep_df = pandas.DataFrame(columns=sim_sweep_columns)
print('Got BitFusion Eyeriss, Numbers')

# 执行仿真器的扫描
ps4_e_results = check_pandas_or_run(ps4_e_sim, ps4_e_sim_sweep_df,
        ps4_e_sim_sweep_csv, batch_size=TW, list_bench=benchmarks.snn_benchlist)

In [ ]:
sim_sweep_columns = ['N', 'M',
        'Max Precision (bits)', 'Min Precision (bits)',
        'Network', 'Layer',
        'Cycles', 'Memory wait cycles',
        'WBUF Read', 'WBUF Write',
        'OBUF Read', 'OBUF Write',
        'IBUF Read', 'IBUF Write',
        'DRAM Read', 'DRAM Write',
        'Bandwidth (bits/cycle)',
        'WBUF Size (bits)', 'OBUF Size (bits)', 'IBUF Size (bits)',
        'Batch size', 'in_sparsity_util', 'out_sparsity_util']

# 向PS4_e_results中插入PTB周期、PS4周期、PS4投机周期和PS4恢复周期
# create a new column {valid_cycles} to store the df['Cycles'] * (1 - df['in_sparsity_util'])
ps4_e_results.insert(2, 'PTB-Cycles', ps4_e_results['Cycles'] * (1 - ps4_e_results['in_sparsity_util']))
# PTB-cycles = cycles * (1 - input_sparsity)
ps4_e_results.insert(3, 'PS4-Cycles', ps4_e_results['Cycles'] * (1 - ps4_e_results['out_sparsity_util'] + ps4_e_results['Batch size'].apply(lambda x: math.log2(x) / x)))
# PS4-cycles = cycles * (1 - output_sparsity + log(batch_size)/batch_size)
ps4_e_results.insert(4, 'PS4-Spec-Cycles', ps4_e_results['Cycles'] * (ps4_e_results['Batch size'].apply(lambda x: math.log2(x) / x)))
# PS4-speculative-sycles投机周期 = cycles * log(batch_size)/batch_size
ps4_e_results.insert(5, 'PS4-Rec-Cycles', ps4_e_results['Cycles'] * (1 - ps4_e_results['out_sparsity_util']))
# PS4-recovery-sycles恢复周期 = cycles * (1 - output_sparsity)
ps4_e_results = ps4_e_results.groupby(['N', 'M', 'Batch size', 'Network'],as_index=False)
# 将结果根据PE数、batchsize(TW)和网络类型合并整理

In [ ]:
ps4_e_results.head()

## Layer-wise analysis
画出某个网络中不同层之间的投机周期数、恢复周期数、总周期数、稀疏率等

In [ ]:
# df_imagenet = ps4_e_results.get_group((128, 2, TW, 'ImageNetSNN'))
# 取结果中的一组数据
df_imagenet = ps4_e_results.get_group((128, 2, TW, 'DVS128Gesture'))
df_imagenet.head()

In [ ]:
from cProfile import label
from click import style
from cv2 import rotate
from matplotlib import markers
from matplotlib.lines import lineStyles


fig, ax = plt.subplots(figsize=(8, 4))
ln1 = ax.plot(df_imagenet['Layer'].apply(lambda x: x.split('/')[0]).to_list() + ['Geomean'], (df_imagenet['Cycles'] / df_imagenet['Cycles']).to_list() + [np.mean(df_imagenet['Cycles'] / df_imagenet['Cycles'])], label='Baseline', marker='o', linestyle='-', color=color_bar1[0], markersize=8, linewidth=2)

ln2 = ax.plot(df_imagenet['Layer'].apply(lambda x: x.split('/')[0]).to_list() + ['Geomean'], (df_imagenet['PS4-Spec-Cycles'] / df_imagenet['Cycles']).to_list() + [np.mean(df_imagenet['PS4-Spec-Cycles'] / df_imagenet['Cycles'])], label='SFF', marker='>', linestyle='-', color=color_bar1[1], markersize=8, linewidth=2)
plt.text(.3+df_imagenet.shape[0], 0.03 + np.mean(df_imagenet['PS4-Spec-Cycles'] / df_imagenet['Cycles']), '{:.2%}'.format(np.mean(df_imagenet['PS4-Spec-Cycles'] / df_imagenet['Cycles'])), ha='right', va='center', color=color_bar1[1], fontsize=14)

ln3 = ax.plot(df_imagenet['Layer'].apply(lambda x: x.split('/')[0]).to_list() + ['Geomean'], (df_imagenet['PS4-Rec-Cycles'] / df_imagenet['Cycles']).to_list() + [np.mean(df_imagenet['PS4-Rec-Cycles'] / df_imagenet['Cycles'])], label='ER', marker='<', linestyle='-', color=color_bar1[2], markersize=8, linewidth=2)
plt.text(.3+df_imagenet.shape[0], 0.03 + np.mean(df_imagenet['PS4-Rec-Cycles'] / df_imagenet['Cycles']), '{:.2%}'.format(np.mean(df_imagenet['PS4-Rec-Cycles'] / df_imagenet['Cycles'])), ha='right', va='center', color=color_bar1[2], fontsize=14)

ln4 = ax.plot(df_imagenet['Layer'].apply(lambda x: x.split('/')[0]).to_list() + ['Geomean'], (df_imagenet['PS4-Cycles'] / df_imagenet['Cycles']).to_list() + [np.mean(df_imagenet['PS4-Cycles'] / df_imagenet['Cycles'])], label='PS4', marker='*', linestyle='-', color=color_bar1[3], markersize=8, linewidth=2)
plt.text(.3+df_imagenet.shape[0], 0.07 + np.mean(df_imagenet['PS4-Cycles'] / df_imagenet['Cycles']), '{:.2%}'.format(np.mean(df_imagenet['PS4-Cycles'] / df_imagenet['Cycles'])), ha='right', va='center', color=color_bar1[3], fontsize=14)

plt.xticks(rotation=45)
plt.ylabel('Normalized Cycles')
plt.xlabel("Layers of DVS128Gesture")
ax.xaxis.set_label_coords(.5, -0.26)
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y))) 
ax2 = plt.twinx()
ln5 = ax2.plot(df_imagenet['Layer'].apply(lambda x: x.split('/')[0]).to_list() + ['Geomean'], (df_imagenet['out_sparsity_util']).to_list() + [np.mean(df_imagenet['out_sparsity_util'])], label='sparsity_util', marker='+', linestyle='--', color='black', markersize=8, linewidth=2)

print(np.mean(df_imagenet['out_sparsity_util']))
# annotate mean out_sparsity_util
plt.text(.3+df_imagenet.shape[0], -0.03 + np.mean(df_imagenet['out_sparsity_util']), '{:.2%}'.format(np.mean(df_imagenet['out_sparsity_util'])), ha='right', va='center', color='black', fontsize=14)
ax2.set_ylabel('Output Sparsity Utilization')
ax2.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y))) 

lns = ln1 + ln2 + ln3 + ln4 + ln5
labels = [l.get_label() for l in lns]
plt.legend(lns, labels, fontsize=14, loc='upper center', ncol=5, bbox_to_anchor=(0.5, 1.2), fancybox=True, shadow=True, labelspacing=0.2, handleheight=1.6, handlelength=1.2)
plt.savefig(os.path.join(fig_dir, 'ImageNet-16-cyclesbrk.pdf'), bbox_inches='tight')

===================================================================================================================

In [ ]:
# 对结果的每一列进行求和
ps4_e_results = ps4_e_results.agg(np.sum)

In [ ]:
# 计算相较于PTB和baseline的加速比例（计算时钟周期）
ps4_e_results.insert(6, 'speedup_vs_ptb', ps4_e_results['PTB-Cycles'] / ps4_e_results['PS4-Cycles'])
ps4_e_results.insert(7, 'speedup_vs_base', ps4_e_results['Cycles'] / ps4_e_results['PS4-Cycles'])

In [ ]:
ps4_e_results.head()

## Speedup
绘制加速比柱状图

In [ ]:
fig, ax = plt.subplots(figsize=(8, 3))
bar_width = 0.2
index = np.arange(3)
mode_indexes = []

unique_mode = ['Baseline', 'PTB', 'PS4']

speedup = []

lns = []
# mode: Baseline
for idx, mode in enumerate(unique_mode):
    if 'Speedup' in ps4_e_results.columns:  # 如果存在speedup列，删除
        ps4_e_results.drop(columns=['Speedup'], inplace=True)
    # 计算baseline与这三种模型比较的加速比
    if mode == 'Baseline':
        ps4_e_results.insert(0, 'Speedup', ps4_e_results['Cycles'] / ps4_e_results['Cycles'])
    elif mode == 'PTB':
        ps4_e_results.insert(0, 'Speedup', ps4_e_results['Cycles'] / ps4_e_results['PTB-Cycles'])
    elif mode == 'PS4':
        ps4_e_results.insert(0, 'Speedup', ps4_e_results['Cycles'] / ps4_e_results['PS4-Cycles'])
        speedup = ps4_e_results['Speedup']
    else:
        raise ValueError('Invalid mode')
    
    # 以三个模型分组并绘制柱状图
    res = ps4_e_results[['Network', 'Speedup']]
    res.sort_values(by='Network', inplace=True)
    bottom = np.zeros(3)
    plt.bar(index + (idx - 1) * bar_width, res.iloc[:, 1], bar_width, bottom=bottom, label=mode)
    bottom += res.iloc[:, 1]
    mode_indexes.append(index + (idx - 1) * bar_width)

plt.xticks([])
plt.legend(["Baseline", "PTB", "PS4"], fontsize=14, loc='upper center', bbox_to_anchor=(0.5, 1.22),
                 fancybox=True, shadow=True, ncol=3, labelspacing=1.0, handleheight=1.0, columnspacing=6)

dataset_names = sorted(ps4_e_results['Network'].unique())
# replace "CIFAR10SNN" with "CIFAR10DVS" in dataset_names
# replace "ImageNetSNN" with "ImageNet" in dataset_names

for i, dataset_name in enumerate(dataset_names):
    if dataset_name == 'CIFAR10SNN':
        dataset_name = 'CIFAR10DVS'
    elif dataset_name == 'ImageNetSNN':
        dataset_name = 'ImageNet'
    plt.annotate(dataset_name, (index[i], 0), textcoords="offset points", xytext=(0, -15), ha='center', fontsize=14)

# plt.axhline(y=1, xmin=0.4, xmax=0.7, color='black', linestyle='--', linewidth=1)
plt.axhline(speedup.iloc[1], xmin=0.4, xmax=0.6, color='black', linestyle='--', linewidth=1)
plt.annotate("", xy=(0.8, 1.02), xytext=(0.8, 1*speedup.iloc[1]), color="#a30b33", arrowprops=dict(arrowstyle="<->", color="#a30b33", linewidth=2))
plt.annotate(f"{speedup.iloc[1]:.2f}x speedup\nw.r.t Baseline", xy=(0.9, 1.2), xytext=(0.9, 0.7*speedup.iloc[1]), color="#a30b33", fontsize=14)

plt.ylabel("Speedup")
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y))) 
print(speedup)
plt.savefig('fig/speedup.pdf', bbox_inches='tight')

## Energy Efficiency

In [ ]:
breakdown_dict = {}
for mode in ['Baseline', 'PTB', 'PS4']:
    breakdown_data = []     # ['core', 'buffer', 'dram', 'popcnt']
    for bench in benchmarks.snn_benchlist:
        ps4_e_stats = df_to_stats(ps4_e_results.loc[ps4_e_results['Network'] == bench])  # 转换到stats
        ps4_breakdown = ps4_e_stats.get_energy_breakdown(ps4_e_sim.get_energy_cost(syn_fn), mode=mode)  # 获得每个功能部分的能耗
        total = sum(ps4_breakdown)  # 求和
        ps4_breakdown = [x / float(total) for x in ps4_breakdown]  # 计算占比

        breakdown_data.append([mode, bench, total] + ps4_breakdown)
    breakdown_dict[mode] = breakdown_data

breakdown_dict.keys()


In [ ]:
fig, ax = plt.subplots(figsize=(8, 3))
bar_width = 0.2
index = np.arange(3)
mode_indexes = []

unique_mode = ['Baseline', 'PTB', 'PS4']
unique_energy_key = ['core', 'buffer', 'dram', 'popcnt']

speedup = []

lns = []
# mode: Baseline
for idx, mode in enumerate(unique_mode):
    bottom = np.zeros(3)
    # plot total energy for each mode, which normalized to Baseline
    plt.bar(index + (idx - 1) * bar_width, np.array(breakdown_dict[mode])[:, 2].astype(float) / np.array(breakdown_dict['Baseline'])[:, 2].astype(float), bar_width, bottom=bottom, label=mode)
    if mode == 'PS4':
        ee = np.array(breakdown_dict[mode])[:, 2].astype(float) / np.array(breakdown_dict['Baseline'])[:, 2].astype(float)
        plt.text(index[0] + (idx - 1) * bar_width, 0.9, f'{ee[0]:.2f}x', ha='center', va='bottom', fontsize=14)
        plt.text(index[1] + (idx - 1) * bar_width, 0.9, f'{ee[1]:.2f}x', ha='center', va='bottom', fontsize=14)
        plt.text(index[2] + (idx - 1) * bar_width, 0.9, f'{ee[2]:.2f}x', ha='center', va='bottom', fontsize=14)

plt.xticks([])
plt.legend(["Baseline", "PTB", "PS4"], fontsize=14, loc='upper center', bbox_to_anchor=(0.5, 1.22),
                 fancybox=True, shadow=True, ncol=3, labelspacing=1.0, handleheight=1.0, columnspacing=6)

dataset_names = sorted(ps4_e_results['Network'].unique())
for i, dataset_name in enumerate(dataset_names):
    if dataset_name == 'CIFAR10SNN':
        dataset_name = 'CIFAR10DVS'
    elif dataset_name == 'ImageNetSNN':
        dataset_name = 'ImageNet'
    plt.annotate(dataset_name, (index[i], 0), textcoords="offset points", xytext=(0, -15), ha='center', fontsize=14)


plt.ylabel("Energy Efficiency")
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y))) 
plt.savefig('fig/energy_efficiency.pdf', bbox_inches='tight')

## Energy Breakdown

In [ ]:
fig, ax = plt.subplots(figsize=(8, 3))
bar_width = 0.2
index = np.arange(3)
mode_indexes = []
hatches = ['x', '..', '\\', '//', 'o', 'O', '.', 'x']
unique_mode = ['Baseline', 'PTB', 'PS4']
unique_energy_key = ['core', 'buffer', 'dram', 'popcnt']

speedup = []

lns = []
# mode: Baseline
for idx, mode in enumerate(unique_mode):
    bottom = np.zeros(3)
    print(breakdown_dict[mode])
    for idk, unique_key in enumerate(unique_energy_key):
        if idx == 0:
            lns.append(plt.bar(index + (idx - 1) * bar_width, np.array(breakdown_dict[mode])[:, idk+3].astype(float), bar_width, bottom=bottom, label=unique_key, color=color_bar4[idx], hatch=hatches[idk], edgecolor='black'))
        else:
            plt.bar(index + (idx - 1) * bar_width, np.array(breakdown_dict[mode])[:, idk+3].astype(float), bar_width, bottom=bottom, color=color_bar4[idx], hatch=hatches[idk], edgecolor='black')
        bottom += np.array(breakdown_dict[mode])[:, idk+3].astype(float)

        if idk == 0:
            mode_indexes.append(
                index + (idx - (math.floor(3))/2)*bar_width)

plt.xticks([])
plt.xticks(list(np.array(mode_indexes).T.flatten()+0.1), unique_mode * 3, rotation=45, fontsize=14)
labs = [l.get_label() for l in lns]
plt.legend(lns, labs, fontsize=14, loc='upper center', bbox_to_anchor=(0.5, 1.22),
                 fancybox=True, shadow=True, ncol=4, labelspacing=1.0, handleheight=1.0, columnspacing=3)


dataset_names = sorted(ps4_e_results['Network'].unique())
for i, dataset_name in enumerate(dataset_names):
    if dataset_name == 'CIFAR10SNN':
        dataset_name = 'CIFAR10DVS'
    elif dataset_name == 'ImageNetSNN':
        dataset_name = 'ImageNet'
    plt.annotate(dataset_name, (index[i], 0), textcoords="offset points", xytext=(0, -60), ha='center', fontsize=14)

plt.ylabel("Energy Efficiency")
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y))) 
plt.savefig('fig/energy_breakdown.pdf', bbox_inches='tight')

## TW sensitivity analysis

In [ ]:
df = pd.read_csv('results/ps4-sim-sweep.csv')
df.head()

In [ ]:
df.insert(3, 'PS4-Cycles', df['Cycles'] * (1 - df['out_sparsity_util'] + df['Batch size'].apply(lambda x: math.log2(x) / x)))
df.insert(6, 'speedup_vs_base', df['Cycles'] / df['PS4-Cycles'])

In [ ]:
df['Network'].unique()

In [ ]:
df_grp = df.groupby(['N', 'M', 'Batch size', 'Network'],as_index=False)
df_grp.head()

In [ ]:
dataset = 'DVS128Gesture'
df_grp.get_group((128, 2, 8, dataset))

In [ ]:
N = df['N'].unique()
M = df['M'].unique()
TW = df['Batch size'].unique()
# 遍历df groups
print(len(list(product(zip(N, M), TW))))
failed = 0
for (n, m), tw in product(zip(N, M), TW):
    try:
        series = df_grp.get_group((n, m, tw, dataset))
    except:
        print(f"n={n}, m={m}, tw={tw} not found")
        failed += 1
print(failed)

In [ ]:
speedup = [df_grp.get_group((128, 2, tw, dataset))['speedup_vs_base'].values[0] for tw in TW]
out_sparsity_utils = [df_grp.get_group((128, 2, tw, dataset))['out_sparsity_util'].values[0] for tw in TW]

In [ ]:
speedup

In [ ]:
out_sparsity_utils

In [ ]:
M

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3))
bar_width = 0.3
bottom = np.zeros(len(TW))
index = np.arange(len(TW))
ln1 = plt.bar(index, speedup, bar_width, bottom=bottom, label='Speedup', color=color_bar4[2])
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y))) 
plt.xlabel('TW', fontsize=14)
plt.ylabel('Speedup')
ymin, ymax = ax.get_ylim()
ax.set_ylim(ymin, 4)
ax2 = plt.twinx()
ln2 = ax2.plot(index, out_sparsity_utils, label='out_sparsity_util', marker='o', linestyle='-', color=color_bar4[5], markersize=8, linewidth=2)
ax2.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y))) 
plt.xticks(index, TW)
plt.ylabel('Output Sparsity Util.')
ax2.set_ylim(0.8, 1.0)
# add legend for ax1 and ax2
plt.legend(["out_sparsity_util"], fontsize=14, loc='upper center', bbox_to_anchor=(0.3, 1.0),
                 fancybox=True, shadow=True, ncol=2, labelspacing=1.0, handleheight=0.8, columnspacing=6)
plt.savefig('fig/TW_sensitivity.pdf', bbox_inches='tight')

In [ ]:
# speedup = [df_grp.get_group((32, 32, tw, 'DVS128Gesture'))['speedup_vs_base'].values[0] for tw in TW]
speedup = [df_grp.get_group((n, m, 8, dataset))['speedup_vs_base'].values[0] for n, m in zip(N, M)]
out_sparsity_utils = [df_grp.get_group((n, m, 8, dataset))['out_sparsity_util'].values[0] for n, m in zip(N, M)]


In [ ]:
speedup

In [ ]:
out_sparsity_utils

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3))
bar_width = 0.3
bottom = np.zeros(len(M))
index = np.arange(len(M))
ln1 = plt.bar(index, speedup, bar_width, bottom=bottom, label='Speedup', color=color_bar4[2])
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y))) 
plt.xlabel('# of column', fontsize=14)
plt.ylabel('Speedup')
ax2 = plt.twinx()
ln2 = ax2.plot(index, out_sparsity_utils, label='out_sparsity_util', marker='o', linestyle='-', color=color_bar4[5], markersize=8, linewidth=2)
ax2.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y))) 
plt.xticks(index, M)
plt.ylabel('Output Sparsity Util.')
ymin, ymax = ax2.get_ylim()
ax2.set_ylim(ymin, 1.0)
# add legend for ax1 and ax2
plt.legend(["out_sparsity_util"], fontsize=14, loc='upper center', bbox_to_anchor=(0.72, 1.1),
                 fancybox=True, shadow=True, ncol=2, labelspacing=1.0, handleheight=0.8, columnspacing=6)
plt.savefig('fig/col_sensitivity.pdf', bbox_inches='tight')

In [ ]:
config_file = 'ps4_e_conf_best.ini'

batch_size = 100

TW = 48

syn_fn = 'systolic_array_synth_snn.csv'

# Create simulator object
verbose = False
ps4_e_sim = Simulator(config_file, verbose)
ps4_e_energy_costs = ps4_e_sim.get_energy_cost(syn_fn)
# ps4_e_energy_costs = ps4_e_sim.get_energy_cost()
print(ps4_e_sim)

energy_tuple = ps4_e_energy_costs
print('')
print('*'*50)
print(energy_tuple)

ps4_e_sim_sweep_csv = os.path.join(results_dir, 'ps4-sim-sweep.csv')
# if os.path.exists(ps4_e_sim_sweep_csv):
#     os.remove(ps4_e_sim_sweep_csv)
if os.path.exists(ps4_e_sim_sweep_csv):
    ps4_e_sim_sweep_df = pandas.read_csv(ps4_e_sim_sweep_csv)
else:
    ps4_e_sim_sweep_df = pandas.DataFrame(columns=sim_sweep_columns)
print('Got BitFusion Eyeriss, Numbers')

ps4_e_results = check_pandas_or_run(ps4_e_sim, ps4_e_sim_sweep_df,
        ps4_e_sim_sweep_csv, batch_size=TW, list_bench=benchmarks.snn_benchlist)

ps4_e_results.insert(2, 'PTB-Cycles', ps4_e_results['Cycles'] * (1 - ps4_e_results['in_sparsity_util']))
ps4_e_results.insert(3, 'PS4-Cycles', ps4_e_results['Cycles'] * (1 - ps4_e_results['out_sparsity_util'] + ps4_e_results['Batch size'].apply(lambda x: math.log2(x) / x)))
ps4_e_results.insert(4, 'PS4-Spec-Cycles', ps4_e_results['Cycles'] * (ps4_e_results['Batch size'].apply(lambda x: math.log2(x) / x)))
ps4_e_results.insert(5, 'PS4-Rec-Cycles', ps4_e_results['Cycles'] * (1 - ps4_e_results['out_sparsity_util']))

ps4_e_results = ps4_e_results.groupby('Network',as_index=False).agg(np.sum)
ps4_e_results.head()

In [ ]:
ps4_e_results["Cycles"]

In [ ]:
breakdown_dict = {}
for mode in ['Baseline', 'PTB', 'PS4']:
    breakdown_data = []     # ['core', 'buffer', 'dram', 'popcnt']
    for bench in benchmarks.snn_benchlist:
        print(f"{bench}")
        ps4_e_stats = df_to_stats(ps4_e_results.loc[ps4_e_results['Network'] == 'DVS128Gesture'])
        ps4_breakdown = ps4_e_stats.get_energy_breakdown(ps4_e_sim.get_energy_cost(syn_fn), mode=mode)
        total = sum(ps4_breakdown)
        if mode == 'PS4':
            print(f"total energy = {total}")
        ps4_breakdown = [x / float(total) for x in ps4_breakdown]

        breakdown_data.append([mode, bench, total] + ps4_breakdown)
    breakdown_dict[mode] = breakdown_data

breakdown_dict.keys()
